# Running SV calling

## February 2020
### based on script from Jinhui Ding (CBG)

In [ ]:
##setting up environment
import pandas as pd

WRKDIR = '/home/mooreank/sv-manta'

In [21]:

##Manta SV
WRKDIR='/home/mooreank/WGS_sample_processing'
PROJECT_ID='nih-nia-lng-cbg'
COHORT='UNHS'
IN_LIST=samples
PIPEYAML='/labshare/Jinhui/to_Anni/Manta_v1.6.yaml'
LABELNAME='manta'
LCCOHORT = f'{COHORT}'.lower()


#LCCOHORT=$(echo ${COHORT} | awk '{ print tolower($1) }')
##now loop over the list file running the Manta yaml per sample in the list file
def writecommands(sample_list, out_file_name):
    out_file = open(f'{out_file_name}', "w")
    for SAMPLE in samples:
        LABEL=f'{SAMPLE} OPID='
        cmd = f'gcloud alpha genomics pipelines run \
    --project {PROJECT_ID}  \
    --pipeline-file {PIPEYAML} \
    --zones us-central1-f \
    --inputs EXECTOOLmanta=gs://nihnia-dementia-wgs/resources/tools/manta-1.6.0/manta-1.6.0.centos6_x86_64.tar.gz \
    --inputs REFSEQ=gs://nihnia-dementia-wgs/resources/broad/hg38/v0/Homo_sapiens_assembly38.fasta \
    --inputs REFSEQINDEX=gs://nihnia-dementia-wgs/resources/broad/hg38/v0/Homo_sapiens_assembly38.fasta.fai \
    --inputs INCRAM=gs://nihnialngcbg-testing/UNHS/crams/{SAMPLE}/{SAMPLE}.cram  \
    --inputs INCRAMINDEX=gs://nihnialngcbg-testing/UNHS/crams/{SAMPLE}/{SAMPLE}.cram.crai \
    --outputs OUTSUMMARY=gs://nihnialngcbg-testing/UNHS/sv-manta/{SAMPLE}/{SAMPLE}.alignmentStatsSummary.txt \
    --outputs OUTSMALLINDELS=gs://nihnialngcbg-testing/UNHS/sv-manta/{SAMPLE}/{SAMPLE}.candidateSmallIndels.vcf.gz \
    --outputs OUTSMALLINDELSINDEX=gs://nihnialngcbg-testing/UNHS/sv-manta/{SAMPLE}/{SAMPLE}.candidateSmallIndels.vcf.gz.tbi \
    --outputs OUTSV=gs://nihnialngcbg-testing/UNHS/sv-manta/{SAMPLE}/{SAMPLE}.candidateSV.vcf.gz \
    --outputs OUTSVINDEX=gs://nihnialngcbg-testing/UNHS/sv-manta/{SAMPLE}/{SAMPLE}.candidateSV.vcf.gz.tbi \
    --outputs OUTDIPLOIDSV=gs://nihnialngcbg-testing/UNHS/sv-manta/{SAMPLE}/{SAMPLE}.diploidSV.vcf.gz \
    --outputs OUTDIPLOIDSVINDEX=gs://nihnialngcbg-testing/UNHS/sv-manta/{SAMPLE}/{SAMPLE}.diploidSV.vcf.gz.tbi \
    --outputs OUTSTATSTSV=gs://nihnialngcbg-testing/UNHS/sv-manta/{SAMPLE}/{SAMPLE}.svCandidateGenerationStats.tsv \
    --outputs OUTSTATSXML=gs://nihnialngcbg-testing/UNHS/sv-manta/{SAMPLE}/{SAMPLE}.svCandidateGenerationStats.xml \
    --outputs OUTGRAPHTSV=gs://nihnialngcbg-testing/UNHS/sv-manta/{SAMPLE}/{SAMPLE}.svLocusGraphStats.tsv \
    --logging gs://nihnialngcbg-testing/UNHS/logs/Manta/{SAMPLE} \
    --labels=pipe=manta,sample={LABELNAME},cohort={LCCOHORT}'

        
        print(f'echo -n {LABEL}', file=out_file)
        print(f'{cmd}\n', file=out_file)


In [22]:

script_file = f'{WRKDIR}/UNHS.manta.run.sh'

writecommands(samples, script_file)

#iterate over samples formatting the cmds

# with open(temp_script_file, 'w') as file_handler:
#         for this_cmd in cmds:
#             file_handler.write("{}\n".format(this_cmd))
            
print('#run these commands at terminal:\n')
print('chmod +x ' + script_file)
print('nohup ' + script_file + ' > {}/{}.UNHS.manta.run.log &'.format(WRKDIR,COHORT.lower()))

################

#run these commands at terminal:

chmod +x /home/mooreank/WGS_sample_processing/UNHS.manta.run.sh
nohup /home/mooreank/WGS_sample_processing/UNHS.manta.run.sh > /home/mooreank/WGS_sample_processing/unhs.UNHS.manta.run.log &


In [ ]:
#the above google genomics pipeline (ggp) commands should return on OPIDs
#grab to previous operation IDs to use here as opids list
opid = 'EJucjt-JLhiA5ans-qTawPMBINn1oK--DSoPcHJvZHVjdGlvblF1ZXVl'
#for opid in opids:
!gcloud alpha genomics operations describe {opid} --format='yaml(done, error, metadata.events)'

In [ ]:
for sample in samples_list:
    cmd = f'gunzip {sample}.diploidSV.vcf.gz'
    print(cmd)

In [ ]:
##used Mantas convertInversion.py script to covert version 1.6 output back to 1.4 to deal w inversion (ran on local)
print('ran in terminal, python 2 environment of local ((needs to run with python2!))')
print('')
for sample in samples_list:
    cmd = f'python convertInversion.py /Users/mooreank/anaconda3/bin/samtools Homo_sapiens_assembly38.fasta {sample}.diploidSV.vcf > {sample}.diploidSV.1.4.vcf'
    print(cmd)

In [ ]:
#ran in local

#For SVTK, https://github.com/talkowski-lab/svtk, here are the major steps to run on local LNG servers:
########
WRKDIR='/Users/mooreank/Desktop/Mark/UNHS/sv-manta'

##problem files
error_list = []
##clean Manta output: 1.select "PASS" calls 2.remove chr, [, ]
for sample in error_list:
    ###keeping all PASS
    cmd = f"""awk '$1 ~ "#" {{ print }} $1 !~ "#"  && $7 == "NoPairedSupport" {{ print }}' {WRKDIR}/{sample}.diploidSV.vcf > {WRKDIR}/{sample}.diploidSV.nopaired.vcf"""
    print(cmd)

    

In [ ]:
#ran in local

#For SVTK, https://github.com/talkowski-lab/svtk, here are the major steps to run on local LNG servers:
########
WRKDIR='/Users/mooreank/Desktop/Mark/UNHS/sv-manta'

##clean Manta output: 1.select "PASS" calls 2.remove chr, [, ]
for sample in samples_list:
    ###keeping all PASS
    cmd = f"""awk '$1 ~ "#" {{ print }} $1 !~ "#"  && $7 == "PASS" {{ print }}' {WRKDIR}/{sample}.diploidSV.1.4.vcf > {WRKDIR}/temp.{sample}.diploidSV.1.4.vcf"""
    print(cmd)
    
    
    
print("")    
    
for sample in samples_list:   
    cmd2 = f"""awk '{{ gsub(/chr/, ""); print}}' {WRKDIR}/temp.{sample}.diploidSV.1.4.vcf > {WRKDIR}/{sample}.diploidSV.1.4.edit.vcf"""
    print(cmd2)
    

In [10]:
#remove temp files
cmd4 = f'rm {WRKDIR}/temp*diploidSV.vcf'
#cmd5 = f'rm {WRKDIR}/temp2.{sample}.diploidSV.vcf'  
!{cmd4}

    ##get rid of chr within brackets, check manta version 1.4/1.6, dont have to rerun if theres no inversions?

In [ ]:
# ##checking for inversions -> found none in any files
# for sample in samples_list:
#     cmd = f"""awk '$5 == "<INV>" {{print}}' {WRKDIR}/{sample}.diploidSV.edit.vcf > {sample}.test.vcf"""
#     print(cmd)
    

In [ ]:
##ran on node 2 

for sample in samples_list:
    ##per sample run svtk standardize
    cmd = f'svtk standardize {sample}.diploidSV.1.4.edit.vcf {sample}.diploidSV.standardize.vcf manta'
    print(cmd)
print('')


In [16]:
##ran on node2

##create a list file with path to the per sample standardize.vcf files

WRKDIR = '/home/mooreank/sv-manta'
out_file_name = f'{WRKDIR}/sv-manta.standardize.vcf.list'

# out_file = open(f'{out_file_name}', "w")
# for sample in samples_list:
    
#     direct = f'{WRKDIR}/{sample}.diploidSV.standardize.vcf'
#     print(f'{direct}', file=out_file)

with open(f'{out_file_name}', "w") as output:
    for sample in samples_list:
        direct = f'{WRKDIR}/{sample}.diploidSV.standardize.vcf'
        output.write(direct + '\n')
    
    
#########     

In [17]:
print('run in terminal:')
print("")
cmd2 = f'svtk vcfcluster -t INS,DEL,DUP,INV,BND {WRKDIR}/sv-manta.standardize.vcf.list sv-manta.standardize.cluster.vcf'
print(cmd2)

run in terminal:

svtk vcfcluster -t INS,DEL,DUP,INV,BND /home/mooreank/sv-manta/sv-manta.standardize.vcf.list sv-manta.standardize.cluster.vcf
